In [85]:
import pandas as pd
import numpy as np
import vk_api
import random
import os

In [6]:
# Task 1
# Прочитайте csv (ads_data_121288 - ads_data_121288.csv) файл, который находится в этой папке, при помощи библиотеки pandas

In [4]:
path = 'https://raw.githubusercontent.com/sborisv74/Airflow_tests/main/ads_data_121288.csv'
ads = pd.read_csv(path)

In [5]:
ads

,date,time,event,platform,ad_id,ad_cost_type,ad_cost
0,2019-04-01,2019-04-01 0:00:48,view,web,121288,CPM,187.4
1,2019-04-01,2019-04-01 0:04:41,view,ios,121288,CPM,187.4
2,2019-04-01,2019-04-01 0:07:50,view,android,121288,CPM,187.4
3,2019-04-01,2019-04-01 0:07:50,view,android,121288,CPM,187.4
4,2019-04-01,2019-04-01 0:08:46,view,ios,121288,CPM,187.4
...,...,...,...,...,...,...,...
602,2019-04-02,2019-04-01 22:39:16,click,android,121288,CPM,187.4
603,2019-04-02,2019-04-01 22:40:29,click,android,121288,CPM,187.4
604,2019-04-02,2019-04-01 22:40:31,click,web,121288,CPM,187.4
605,2019-04-02,2019-04-01 22:44:14,click,ios,121288,CPM,187.4


In [25]:
# Task 2
# В данных вы найдете информацию о событиях, которые произошли с объявлением 121288 за два дня. 
# Рассчитайте следующие метрики в разрезе каждого дня: количество показов, количество кликов, CTR, сумма потраченных денег. 
# То есть для каждой метрики у вас должно получиться две цифры, за 2019-04-01 и 2019-04-02.
# Рассчитать сумму потраченных денег можно по следующей формуле:
# Значение из колонки ad_cost разделить на 1000 и умножить на количество показов объявления.

views_count = ads \
            .query('event == "view"') \
            .groupby('date') \
            .agg(views=('event', 'count'))
views_count

,views
date,
2019-04-01,491
2019-04-02,93


In [26]:
clicks_count = ads \
            .query('event == "click"') \
            .groupby('date') \
            .agg(clicks=('event','count'))
clicks_count

,clicks
date,
2019-04-01,17
2019-04-02,6


In [44]:
# Solution 2
ads[ads['event'] == 'view'].groupby('date').count().reset_index()
ads[ads['event'] == 'click'].groupby('date').count().reset_index()
# End Solution 2

,date,time,event,platform,ad_id,ad_cost_type,ad_cost
0,2019-04-01,17,17,17,17,17,17
1,2019-04-02,6,6,6,6,6,6


In [50]:
# CTR
ads_views_clicks = pd.merge(views_count, clicks_count, on='date').reset_index()
ads_views_clicks['CTR'] = ads_views_clicks['clicks'] / ads_views_clicks['views'] * 100
ads_views_clicks

,date,views,clicks,CTR
0,2019-04-01,491,17,3.462322
1,2019-04-02,93,6,6.451613


In [51]:
# сумма потраченных денег
ads_views_clicks['money_views'] = ads_views_clicks.views * ((ads.ad_cost / 1000).unique()[0])
ads_views_clicks

,date,views,clicks,CTR,money_views
0,2019-04-01,491,17,3.462322,92.0134
1,2019-04-02,93,6,6.451613,17.4282


In [40]:
# Task 3
# Теперь найдите процентную разницу между этими метриками. 
# То есть найдите, насколько процентов каждая метрика увеличилась/уменьшилась 2 апреля по сравнению с 1 апреля.

In [52]:
# Task 4
# Создайте текстовый файл, в котором будет собрана информация о том, 
# какие метрики наблюдаются 2 апреля (представьте, что 2 апреля это сегодня, условно, вы отправляете отчет за сегодня, сравнивая данные со вчера), 
# а также, на сколько процентов они уменьшились по сравнению со вчера. 
# То есть в текстовом файле должен быть список метрик, их значения 2 апреля и процентное отличие от 1 апреля. 
# Вариант текстового отчета:
# Отчет по объявлению 121288 за 2 апреля
# Траты: Х рублей (+Y%)
# Показы: X (+Y%)
# Клики: X (-Y%)
# CTR: X (-Y%)

In [68]:
money_0104 = float(ads_views_clicks[ads_views_clicks['date'] == '2019-04-01']['money_views'])
views_0104 = float(ads_views_clicks[ads_views_clicks['date'] == '2019-04-01']['views'])
clicks_0104 = float(ads_views_clicks[ads_views_clicks['date'] == '2019-04-01']['clicks'])
CTR_0104 = float(ads_views_clicks[ads_views_clicks['date'] == '2019-04-01']['CTR'])

money_0204 = float(ads_views_clicks[ads_views_clicks['date'] == '2019-04-02']['money_views'])
views_0204 = float(ads_views_clicks[ads_views_clicks['date'] == '2019-04-02']['views'])
clicks_0204 = float(ads_views_clicks[ads_views_clicks['date'] == '2019-04-02']['clicks'])
CTR_0204 = float(ads_views_clicks[ads_views_clicks['date'] == '2019-04-02']['CTR'])

C:\Users\sbori\AppData\Local\Temp\ipykernel_15576\2389568196.py:1: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  money_0104 = float(ads_views_clicks[ads_views_clicks['date'] == '2019-04-01']['money_views'])
C:\Users\sbori\AppData\Local\Temp\ipykernel_15576\2389568196.py:2: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  views_0104 = float(ads_views_clicks[ads_views_clicks['date'] == '2019-04-01']['views'])
C:\Users\sbori\AppData\Local\Temp\ipykernel_15576\2389568196.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  clicks_0104 = float(ads_views_clicks[ads_views_clicks['date'] == '2019-04-01']['clicks'])
C:\Users\sbori\AppData\Local\Temp\ipykernel_15576\2389568196.py:4: FutureWarning: Calling fl

In [86]:
message_vk = f'''Отчет по объявлению 121288 за 2 апреля:\n
Траты: {round(money_0204, 2)} ({round((money_0204 - money_0104)/money_0104 * 100)} %)
Показы: {views_0204} ({round((views_0204 - views_0104)/views_0104 * 100)} %)
Клики: {clicks_0204} ({round((clicks_0204 - clicks_0104)/clicks_0104 * 100)} %)
CTR: {round(CTR_0204, 2)} ({round((CTR_0204 - CTR_0104)/CTR_0104 * 100)} %)
'''

In [90]:
with open(f'report_2019-04-02.txt', 'w', encoding='utf-8') as fwr:
    fwr.write(message_vk)